In [1]:
import os

from rag_tool import construct_rag_tool

from tavily_utils import initialize_web_search_tool
from tavily_utils import execute_web_search

from router import router_tool

from llm_utils import initialize_llm

from agents import initialize_router_agent 
from agents import initialize_retriever_agent
from agents import initialize_answer_grader_agent
from agents import initialize_grader_agent
from agents import initialize_hallucination_grader_agent

from tasks import initialize_router_task
from tasks import initialize_answer_task
from tasks import initialize_retriever_task
from tasks import initialize_grader_task
from tasks import initialize_hallucination_task

from crew_operations import orchestrate_work_flow
from crew_operations import execute_work_flow

from config import GROQ_API_KEY
from config import TAVILY_API_KEY

### Set API Environment Variables

In [2]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

### RAG Tool

In [3]:
'''
Input Parameters to RAG Tool

- URL: https://aclanthology.org/2020.wmt-1.52.pdf
'''

rag_tool = construct_rag_tool(urls=["https://aclanthology.org/2020.wmt-1.52.pdf"])

/Users/apple/miniconda3/envs/venv_langchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


In [4]:
'''
Infer from the RAG Tool
'''
rag_tool.run("Describe the hindi - marathi translation model?")

Using Tool: Search a PDF's content


'Relevant Content:\nCommonCrawl n-grams raw monolingual ﬁles are processed1to remove sentences with invalid char- acters, strip leading and trailing whitespaces, and remove duplicate sentences. 3 System Overview Our Hindi-Marathi primary system is an ensem- ble of two transformer models. One is back- translated model and the other model is trained on anonymized data. 1https:/github.com/kpu/preprocess\n\nProceedings of the 5th Conference on Machine Translation (WMT) , pages 437–441 Online, November 19–20, 2020. c⃝2020 Association for Computational Linguistics437Infosys Machine Translation System for WMT20 Similar Language Translation Task Kamalkumar Rathinasamy, Amanpreet Singh, Balaguru Sivasambagupta, Prajna Prasad Neerchal, Vani Sivasankaran Infosys Limited {kamalkumar r,amanpreet.singh04,balaguru.s,prajna.neerchal,vani.s }@infosys.com Abstract This paper describes Infosys’ submission to the WMT20 Similar Language Translation shared task. We participated in Indo-Aryan language pair i

### Web Search Tool

In [5]:
'''
Initialize Web search tool using Tavily
'''
web_search_tool = initialize_web_search_tool()

In [6]:
'''
Execute web search for the specified content
'''
execute_web_search(web_search_tool=web_search_tool, question="Which is the best World Machine Translation model built in 2020?")

[{'url': 'https://ai.meta.com/blog/introducing-many-to-many-multilingual-machine-translation/',
  'content': "October 19, 2020. Share on Facebook. Share on Twitter. Facebook AI is introducing, M2M-100 the first multilingual machine translation (MMT) model that translates between any pair of 100 languages without relying on English data. It's open sourced here. When translating, say, Chinese to French, previous best multilingual models train on ..."},
 {'url': 'https://slator.com/inside-deepl-the-worlds-fastest-growing-most-secretive-machine-translation-company/',
  'content': 'The Cologne, Germany-based machine translation (MT) provider has since added 21 more languages; first Russian and Portuguese in May 2018, then Chinese and Japanese in March 2020, and a further 13 European languages in March 2021.. Its success in Japan has been particularly rapid, taking less than 17 months from release to becoming the 30th most popular website in the country, according to ...'},
 {'url': 'https:/

### Agentic RAG Design Pattern

#### Large Language Model initialization

In [7]:
llm = initialize_llm()

#### Agents Construction

In [8]:
Router_Agent = initialize_router_agent(llm=llm)
Answer_Grader_Agent = initialize_answer_grader_agent(llm=llm)
Grader_Agent = initialize_grader_agent(llm=llm)
Hallucination_Grader_Agent = initialize_hallucination_grader_agent(llm=llm)
Retreiver_Agent = initialize_retriever_agent(llm=llm)

#### Tasks Construction

In [9]:
Router_Task = initialize_router_task(agent=Router_Agent, tool=router_tool)
Retreiver_Task = initialize_retriever_task(agent=Retreiver_Agent, task=Router_Task)
Grader_Task = initialize_grader_task(agent=Grader_Agent, task=Retreiver_Task)
Hallucination_Task = initialize_hallucination_task(agent=Hallucination_Grader_Agent, task=Grader_Task)
Answer_Task = initialize_answer_task(agent=Answer_Grader_Agent, task=Hallucination_Task)


#### CREW AI Operations

In [10]:
inputs = {"question":"Describe the hindi - marathi translation model?"}

rag_crew = orchestrate_work_flow(agents=[Router_Agent, Retreiver_Agent, Grader_Agent, Hallucination_Grader_Agent, Answer_Grader_Agent],
    tasks=[Router_Task, Retreiver_Task, Grader_Task, Hallucination_Task, Answer_Task])
result = execute_work_flow(rag_crew=rag_crew, inputs=inputs)

print("Final Result extracted from the Agentic RAG")
print(result)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Describe the hindi - marathi translation model?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about a translation model, which is a concept related to Retrieval-Augmented Generation.

Action: vectorstore
Action Input: {"question": "Describe the hindi - marathi translation model"} 

Action 'vectorstore' don't exist, these are the only available Actions: router_tool: router_tool() -    Args:         question (str): The question or query that needs to be routed.      Returns:         str: The name of the tool to be used ('vectorstore' or 'web_search').   

Thought: I should use the router 